# Compound Receptive Fields

From the elementary receptive fields of blurs and convolutions, we can make compound receptive fields by their combination.

In [ ]:
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

# work from project root for local imports
import os
import sys
import subprocess
from pathlib import Path

# root here refers to the segmentron-master folder
root_dir = Path(subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode("utf-8"))
root_dir = root_dir / "segmentron-master"
os.chdir(root_dir)
sys.path.append(str(root_dir))

from sigma.blur import gauss2d_sphere
from sigma.blur import blur2d_sphere
from sigma.blur import dog1d, dog2d

torch.manual_seed(1337)

## Multi-scale by Cascade Smoothing

**Cascade smoothing** is the property of Gaussians that the variance of the composition of Gaussians is the sum of their variances.
That is, for $G_\circ = G_1 \circ G_2$ the variance obeys $\sigma_\circ^2 = \sigma_1^2 + \sigma_2^2$.

For multi-scale processing, we could filter in parallel with blurs for each scale or we could harness cascade smoothing to filter in sequence with smaller blurs.

In [ ]:
# center-surround has little(r) and bigg(er) blurs
lil_sigma = torch.tensor(3.)
big_sigma = torch.tensor(5.)
# cascade smoothing calculates the large blur from the center blur
# by solving for the variance that sums to the large variance
cascade_sigma = (big_sigma.pow(2) - lil_sigma.pow(2)).pow(0.5)

print("sigmas: ", lil_sigma, big_sigma, cascade_sigma)

In [ ]:
x = torch.randn(1, 1, 128, 128)

x_lil = blur2d_sphere(x, lil_sigma)
x_big = blur2d_sphere(x, big_sigma)
x_cascade = blur2d_sphere(x_lil, cascade_sigma)

plt.figure(figsize=(8, 2))
plt.subplot(1, 4, 1)
plt.title('signal')
plt.imshow(x.detach().squeeze().numpy())
plt.axis('off')
plt.subplot(1, 4, 2)
plt.title('small')
plt.imshow(x_lil.detach().squeeze().numpy())
plt.axis('off')
plt.subplot(1, 4, 3)
plt.title('large')
plt.axis('off')
plt.imshow(x_big.detach().squeeze().numpy())
plt.subplot(1, 4, 4)
plt.title('cascade')
plt.imshow(x_cascade.detach().squeeze().numpy())
plt.axis('off')

cascade_mse = (x_big - x_cascade).pow(2).mean().item()
print(f"difference between direct and cascade is {cascade_mse}")

Let's time the sequential cascade and the parallel center-surround.

In [ ]:
%%timeit
x_lil = blur2d_sphere(x, lil_sigma)
x_cascade = blur2d_sphere(x_lil, cascade_sigma)

In [ ]:
%%timeit
x_lil = blur2d_sphere(x, lil_sigma)
x_big = blur2d_sphere(x, big_sigma)

Oh well, there isn't much difference, at least at these dimensions.
It matters more if the dimensions are larger.

## Center-Surround

**Center-surround** have a smaller center enclosed in a larger surround, and because the center and surround responses have different signs, these receptive fields are sensitive to contrast.
They are objects of study in their own right in visual neuroscience and practical for signal processing as **difference of Gaussians (DoG)** for bandpass filtering.
DoG filtering subtracts a large blur from a small blur, preserving the frequencies in between that range.

A contrastive receptive field made by the difference of local and dilated filters is used to good effect in [Context Contrasted Feature and Gated Multi-scale Aggregation for Scene Segmentation. Ding et al. CVPR'18](http://openaccess.thecvf.com/content_cvpr_2018/papers/Ding_Context_Contrasted_Feature_CVPR_2018_paper.pdf).

Let's inspect DoG filtering.

In [ ]:
impulse = torch.zeros(1, 1, 19)
impulse.view(-1)[impulse.numel() // 2] = 1.
f_dog = dog1d(impulse, lil_sigma, big_sigma)

plt.figure(figsize=(5,5))
plt.title("Difference of Gaussians (DoG)")
plt.plot(f_dog.detach().squeeze().numpy())

Now, in 2D this time.
First let's calculate the DoG kernel ourselves, then compare with the impulse response from our module.

In [ ]:
# determine kernel size to cover +/- 2 sigma s.t. >95% of density is included
half_size = int(max(1, torch.ceil(big_sigma * 2.)))
# always make odd kernel to keep coordinates centered
kernel_size = half_size*2 + 1
# calculate unnormalized density then normalize
coord = torch.linspace(-half_size, half_size, steps=kernel_size)
lil_filter = torch.exp(-coord**2 / (2*lil_sigma**2))
big_filter = torch.exp(-coord**2 / (2*big_sigma**2))
# 2D is product of 1D b.c. this is isotropic
lil_filter = lil_filter.view(-1, 1) @ lil_filter.view(1, -1)
big_filter = big_filter.view(-1, 1) @ big_filter.view(1, -1)
# DoG is the difference of the smaller and large blur
dog_filter = lil_filter / lil_filter.sum() - big_filter / big_filter.sum()
dog_filter -= dog_filter.mean()

impulse = torch.zeros(1, 1, 21, 21)
impulse.view(-1)[impulse.numel() // 2] = 1.

dog_response = dog2d(impulse, lil_sigma, big_sigma)
dog_response -= dog_response.mean()

dog_diff = (dog_filter - dog_response).pow(2).mean() 

print(f"Difference in DoG kernels {dog_diff}")

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("DoG (manual)")
plt.imshow(dog_filter.detach().squeeze().numpy())
plt.subplot(1, 2, 2)
plt.title("dog2d (function)")
plt.imshow(dog_response.detach().squeeze().numpy())

Now let's look at the DoG filter output.

In [ ]:
# show filtered signals
x_center = blur2d_sphere(x, lil_sigma)
x_surround = blur2d_sphere(x, big_sigma)
x_dog = dog2d(x, lil_sigma, big_sigma)

plt.figure(figsize=(8, 2))
plt.subplot(1, 4, 1)
plt.title('signal')
plt.imshow(x.detach().squeeze().numpy())
plt.axis('off')
plt.subplot(1, 4, 2)
plt.title('center')
plt.imshow(x_center.detach().squeeze().numpy())
plt.axis('off')
plt.subplot(1, 4, 3)
plt.title('surround')
plt.axis('off')
plt.imshow(x_surround.detach().squeeze().numpy())
plt.subplot(1, 4, 4)
plt.title('DoG')
plt.imshow(x_dog.detach().squeeze().numpy())
plt.axis('off')